In [1]:
import pandas as pd
from hplib import hplib as hpl

In [92]:
hpsnogeneric=hpl.load_database().head(-6)
weather=pd.read_csv('simulation_data/TRJ-Tabelle.csv').head(15)

In [137]:
ground=hpl.HeatingSystem()
Normheizlast=[]
Vorlauftemperatur=[]
Standort=[]
Model=[]
COP=[]
Manufacturer=[]
Normaußentemperatur=[]
Group=[]
for model in hpsnogeneric['Model']:
    hp=hpl.HeatPump(hpl.get_parameters(model))
    manufacturer=hpl.get_parameters(model).Manufacturer.values[0]
    if hp.group_id==1 or hp.group_id==4:
        i=0
        for normaußentemperatur in weather['T_min_ref']:
            i+=1
            for vorlauftemperatur in range(25,71,1):
                Standort.append(i)
                Model.append(model)
                Group.append(hp.group_id)
                Normaußentemperatur.append(normaußentemperatur)
                Manufacturer.append(manufacturer)
                sim=hp.simulate(normaußentemperatur, vorlauftemperatur,normaußentemperatur)
                if sim['COP']==1:
                    Normheizlast.append(0)
                    COP.append(sim['COP'])
                else:
                    Normheizlast.append(sim['P_th'])
                    COP.append(sim['COP'])
                Vorlauftemperatur.append(vorlauftemperatur)
    elif hp.group_id==2 or hp.group_id==5:
        i=0
        for normaußentemperatur in weather['T_min_ref']:
            i+=1
            for vorlauftemperatur in range(25,70,1):
                Standort.append(i)
                Model.append(model)
                Group.append(hp.group_id)
                Normaußentemperatur.append(normaußentemperatur)
                Manufacturer.append(manufacturer)
                sim=hp.simulate(ground.calc_brine_temp(normaußentemperatur), vorlauftemperatur,normaußentemperatur)
                if sim['COP']==1:
                    Normheizlast.append(0)
                    COP.append(sim['COP'])
                else:
                    Normheizlast.append(sim['P_th'])
                    COP.append(sim['COP'])
                Vorlauftemperatur.append(vorlauftemperatur)
df=pd.DataFrame()
df['Standort']=Standort
df['Manufacturer']=Manufacturer
df['Model']=Model
df['Gruppe']=Group
df['Normaußentemperatur']=Normaußentemperatur
df['Vorlauftemperatur']=Vorlauftemperatur
df['Normheizlast']=Normheizlast
df['COP']=COP
df.loc[(df['Gruppe']==1)|(df['Gruppe']==4),'WP-Kategorie']='Luft/Wasser'
df.loc[(df['Gruppe']==2)|(df['Gruppe']==5),'WP-Kategorie']='Sole/Wasser'
df[['Standort', 'Manufacturer', 'Model','WP-Kategorie', 'Normaußentemperatur',
       'Vorlauftemperatur', 'Normheizlast', 'COP']]
df.to_csv('simulation_data/hp_Normheizlast.csv',index=False)